In [1]:
import tabula
from functools import reduce
import pandas as pd
import numpy as np

In [2]:
# Convert directly without editing, avaiable outputs are CSV, TSV or JSON
tabula.convert_into("NGO FR.pdf", "NGO_Key_Financial_Ratios.csv", output_format="csv", pages=(1,3,5))

In [3]:
# Extract tables into pandas, you can just extract and save or edit before saving
# Our example is for a single table split across 3 pages
# We'll extract each piece then merge into a single one
# Two approaches are demonstrated

# 1st approach: Extract each table to a separate dataframe
df1= tabula.read_pdf('NGO FR.pdf', pages=1, lattice=True)
df2= tabula.read_pdf('NGO FR.pdf', pages=3, lattice=True)
df3= tabula.read_pdf('NGO FR.pdf', pages=5, lattice=True)

# 2nd approach: Extract all tables to a single list
# tables_list= tabula.read_pdf('NGO FR.pdf', pages=(1,3,5), lattice=True, multiple_tables=True)

In [4]:
# Merge 

# 1st approach
dfs_list = [df1, df2, df3]
df_master= reduce(lambda left,right: pd.merge(left,right,how='left',left_index=True, right_index=True), dfs_list)

# 2nd approach
# dfs_list = [tables_list[0], tables_list[1], tables_list[2]]
# df_master= reduce(lambda left,right: pd.merge(left,right,how='left',left_index=True, right_index=True), dfs_list)

In [5]:
df_master.head()

,Ratio,Type,Meaning,Formula,Result,Best,Unnamed: 3,Needs Attention,Avrg By Size of NGO
0,Current Ratio,Liquidity,Will current resources cover current liabilities?,Current Assets,Current Liabilities,NaN,> 2,< 1,NaN
1,NaN,F,NaN,NaN,NaN,#DIV/0!,NaN,NaN,NaN
2,Days of Liquid Net Assets,Liquidity,Are there enough liquid resources to cover typ...,Unrestricted Net Assets,Total Expenses - Bad\rDebt Expense -\rDeprecia...,NaN,> 180\rdays,< 90 days,NaN
3,NaN,F,NaN,NaN,NaN,#DIV/0!,NaN,NaN,NaN
4,Quick Ratio,Liquidity,Will the most liquid assets cover current liab...,Cash + Marketable Securities +\rReceivables,Current Liabilities,NaN,> 1,<.25,NaN


In [6]:
# Edit the table, depending on the complexity of your pdf you may need additional edits

# Activate this block of code if you selected 2nd approach
# headers=df_master.iloc[0]
# df_master=df_master[1:].copy()
# df_master.columns=headers
# df_master.reset_index(drop=True, inplace=True)

df_master.drop(columns=['Best', 'Avrg By Size of NGO'], inplace=True)
# both unnamed and np,nan are included here so the code work with either approach selected
df_master.rename(columns={'Formula':'Numerator', 'Result':'Denominator', 'Unnamed: 3':'Benchmark', np.nan:'Benchmark'}, inplace=True)
df_master.loc[8,('Meaning', 'Benchmark', 'Needs Attention')]= 'Monthly expenses covered by liquid assets', 'N/A', 'N/A'
df_master.dropna(axis=0, inplace=True)
df_master.reset_index(drop=True, inplace=True)

In [7]:
df_master

,Ratio,Type,Meaning,Numerator,Denominator,Benchmark,Needs Attention
0,Current Ratio,Liquidity,Will current resources cover current liabilities?,Current Assets,Current Liabilities,> 2,< 1
1,Days of Liquid Net Assets,Liquidity,Are there enough liquid resources to cover typ...,Unrestricted Net Assets,Total Expenses - Bad\rDebt Expense -\rDeprecia...,> 180\rdays,< 90 days
2,Quick Ratio,Liquidity,Will the most liquid assets cover current liab...,Cash + Marketable Securities +\rReceivables,Current Liabilities,> 1,<.25
3,Days of Cash on Hand,Liquidity,Is there enough cash to cover typical operatin...,Cash + Marketable Securities,Total Expenses - Bad\rDebt Expense -\rDeprecia...,> 90 days,< 30 days
4,Months Of Spending,Liquidity,Monthly expenses covered by liquid assets,Current Assets - Current liabilities\r+Temp re...,Total Expenses - Bad\rDebt Expense -\rDeprecia...,N/A,N/A
5,Operating Margin,Profitability,Do typical operating revenues cover typical\ro...,Current Year Unrestricted Net Assets\r+ Previo...,Unrestricted Revenue\rand Other Support,> +5%,decreasing
6,Net Asset Growth,Profitability,Is profitability improving?,Current Year Total Net Assets -\rPrevious Year...,Previous Year Total\rNet Assets,> +5%,decreasing
7,Contributions Ratio,Solvency,How much does this organization depend on\rdon...,Contributions,Total Revenue,> 10%,> 75%
8,Government Revenue Ratio,Solvency,How much does this organization\rdepend on gov...,Government Revenues,Total Revenue,< 25%,> 75%
9,Debt to Assets,Solvency,What percentage of assets were\rfinanced with ...,Total Debt (Liabilities),Total unrestricted Net\rAssets,< 1,> 2


In [8]:
# Save dataframe to...
df_master.to_csv('Key_Financial_Ratios_NGO.csv', index_label=None)
df_master.to_excel('Key_Financial_Ratios_NGO.xlsx', index_label=None)